In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics import accuracy_score

In [2]:
#Import dataset
dataset=pd.read_csv('train.csv',header=None)
test_data=pd.read_csv('test.csv',header=None) 
label_data=pd.read_csv('test_labels.csv')
# label_data

In [3]:
label_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 1 columns):
e    999 non-null object
dtypes: object(1)
memory usage: 7.9+ KB


In [4]:
#Location x is independent(label) and y is independent(vales)
x=dataset.iloc[:,:1]
y=dataset.iloc[:,1:]


In [5]:
ll=[]
for i in range(len(label_data)):
    if (label_data.iloc[i,0] == 'e'):
        ll.append(0)
    else:
        ll.append(1)
        
        

  

In [6]:
train_data=pd.DataFrame(y)
test_data_df=pd.DataFrame(test_data)

In [7]:
train_data.drop(dataset.columns[[11]], axis = 1, inplace = True) 
test_data_df.drop(dataset.columns[[10]], axis = 1, inplace = True) 

In [8]:
feat=[['s','k','f','x','c','b'],['f','g','y','s'],['n','b','c','g','r','p','u','e','w','y'],['t','f'],['a','l','c','y','f','m','n','p','s'],
      ['a','d','f','n'],['c','w','d'],['b','n'],['k','n','b','h','g','r','o','p','u','e','w','y'],['e','t'],
      ['f','y','k','s'],['f','y','k','s'],['n','b','c','g','o','p','e','w','y'],['n','b','c','g','o','p','e','w','y'],
      ['p','u'],['n','o','w','y'],['n','o','t'],['c','e','f','l','n','p','s','z'],['k','n','b','h','r','o','u','w','y'],
      ['a','c','n','s','v','y'],['g','l','m','p','u','w','d'] ]
     

In [9]:
df=pd.DataFrame(columns=None)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4496 entries, 0 to 4495
Data columns (total 21 columns):
1     4496 non-null object
2     4496 non-null object
3     4496 non-null object
4     4496 non-null object
5     4496 non-null object
6     4496 non-null object
7     4496 non-null object
8     4496 non-null object
9     4496 non-null object
10    4496 non-null object
12    4496 non-null object
13    4496 non-null object
14    4496 non-null object
15    4496 non-null object
16    4496 non-null object
17    4496 non-null object
18    4496 non-null object
19    4496 non-null object
20    4496 non-null object
21    4496 non-null object
22    4496 non-null object
dtypes: object(21)
memory usage: 737.7+ KB


In [28]:
n=len(train_data[1].values)
n

4496

In [10]:
for i in range(21):
    dummies1=pd.get_dummies(train_data.iloc[:,i],prefix='',prefix_sep='')
    dummies2=dummies1.T.reindex(feat[i]).T.fillna(0)
    df=pd.concat([df,dummies2],axis=1,sort=False)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4496 entries, 0 to 4495
Columns: 119 entries, s to d
dtypes: float64(17), uint8(102)
memory usage: 1.0 MB


In [12]:
df2=pd.DataFrame(columns=None)

In [13]:
for i in range(21):
    dummies3=pd.get_dummies(test_data.iloc[:,i],prefix='',prefix_sep='')
    dummies4=dummies3.T.reindex(feat[i]).T.fillna(0)
    df2=pd.concat([df2,dummies4],axis=1,sort=False)

In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 119 entries, s to d
dtypes: float64(72), uint8(47)
memory usage: 608.5 KB


In [15]:
#Putting the first row into label and second into pixels
pixel=pd.DataFrame(df).to_numpy()
label=pd.DataFrame(x).to_numpy()
test_pixel=pd.DataFrame(df2).to_numpy()
test_label=pd.DataFrame(label_data).to_numpy()

label_list=[]
for i in range(0,999):
    lb=test_label[i]
    label_list.append(lb)
# label_list

In [17]:
list2=[]
for i in range(0,999):
    list1=[]
    neighbors = []
    m=test_pixel[i]
    for j in range(1,4469):
        l=pixel[j]
        q=label[j]
        distance = np.linalg.norm(l-m)
        list1.append((q,distance))
        
    list1.sort(key=lambda ele:ele[1])
    k=5
    for p in range(k):
        neighbors.append(list1[p][0])

    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count) 
    list2.append(prediction)

ll=[] 
lll=[]
for i in range(len(label_data)):
    if (label_data.iloc[i,0] == 'e'):
        ll.append(0)
    else:
        ll.append(1)
for i in range(len(list2)):
    if (label_data.iloc[i,0] == 'e'):
        lll.append(0)
    else:
        lll.append(1)
    
            
    
print('The accuracy using euclidean distance with k=5 :')    
print (accuracy_score(ll, lll)*100)   

    
# list2    

The accuracy using euclidean distance with k=5 :
100.0


In [18]:
 list2.info()

AttributeError: 'list' object has no attribute 'info'